In [2]:
%run ../utils.ipynb

[2024-01-10 15:42:37,975 INFO] [numexpr.utils.utils._init_num_threads] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [6]:
from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2024-01-10 15:43:34,411 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2024-01-10 15:43:34,415 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2024-01-10 15:43:34,416 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-01-10 15:43:34,416 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2024-01-10 15:43:34,455 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2024-01-10 15:43:34,457 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2024-01-10 15:43:34,457 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


## Setup

In [8]:
auth_dir = project_dir+"\\auth\\"

In [3]:
League_Ids

{'2015': 853925,
 '2016': 810238,
 '2017': 192551,
 '2018': 799346,
 '2019': 820083,
 '2020': 394682,
 '2021': 364325,
 '2022': 564704,
 '2023': 333146}

In [4]:
Game_Keys

,season,code
16,2015,348
17,2016,359
18,2017,371
19,2018,380
20,2019,390
21,2020,399
22,2021,406
23,2022,414
24,2023,423


In [9]:
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id=333146,
    game_code='nfl',
    game_id=423
)
teams = yahoo_query.get_league_teams()
team_ids_list = []
for team in teams:
    manager = team.managers[0].nickname
    id = team.team_id
    info = {'manager_nickname': manager, 'id': id}
    team_ids_list.append(info)
team_ids_df = pd.DataFrame(team_ids_list)

In [10]:
season_matchups_2023_df = generate_season_matchup_data(yahoo_query,team_ids_df['id'])

Generating season matchup data for team ID 1
Generating season matchup data for team ID 2
Generating season matchup data for team ID 3
Generating season matchup data for team ID 4
Generating season matchup data for team ID 5
Generating season matchup data for team ID 6
Generating season matchup data for team ID 7
Generating season matchup data for team ID 8
Generating season matchup data for team ID 9
Generating season matchup data for team ID 10
Generating season matchup data for team ID 11
Generating season matchup data for team ID 12
Generating season matchup data for team ID 13
Generating season matchup data for team ID 14


In [19]:
yahoo_query.get_team_matchups(14)

[Matchup({
   "is_consolation": 0,
   "is_matchup_recap_available": 1,
   "is_playoffs": 0,
   "is_tied": 0,
   "matchup_grades": [
     {
       "matchup_grade": {
         "grade": "B",
         "team_key": "423.l.333146.t.14"
       }
     },
     {
       "matchup_grade": {
         "grade": "D",
         "team_key": "423.l.333146.t.5"
       }
     }
   ],
   "matchup_recap_title": "who invited canada takes the W against Saquon These Nuts with a solid B grade!",
   "matchup_recap_url": "https://football.fantasysports.yahoo.com/f1/333146/recap?week=1&mid1=14&mid2=5",
   "status": "postevent",
   "teams": [
     {
       "team": {
         "draft_grade": "D-",
         "draft_recap_url": "https://football.fantasysports.yahoo.com/f1/333146/14/draftrecap",
         "faab_balance": 5,
         "has_draft_grade": 1,
         "league_scoring_type": "head",
         "managers": {
           "manager": {
             "felo_score": 712,
             "felo_tier": "gold",
             "guid":

In [86]:
closed_season_dfs = [ season_matchups_2015_df, season_matchups_2016_df, season_matchups_2017_df,
      season_matchups_2018_df, season_matchups_2019_df, season_matchups_2020_df,
      season_matchups_2021_df, season_matchups_2022_df]

In [87]:
to_date_dfs = [ season_matchups_2015_df, season_matchups_2016_df, season_matchups_2017_df,
      season_matchups_2018_df, season_matchups_2019_df, season_matchups_2020_df,
      season_matchups_2021_df, season_matchups_2022_df, season_matchups_2023_df]

In [14]:
df = pd.read_csv(project_dir + "data\\matchup_summary\\closed_seasons_raw.csv")

In [18]:
season_matchups_2023_df.tail(20)

,week,team_name,season_id,manager_name,opponent_team_name,opponent_season_id,opponent_name,projected_points_for,points_for,projected_points_against,points_against
204,13,b'Wereback',13,--hidden--,"b""John Brown's Army""",4,--hidden--,96.20,158.26,87.48,100.56
205,14,b'Wereback',13,--hidden--,b'The fishingtons',8,--hidden--,89.24,74.36,103.54,103.48
206,15,b'Wereback',13,--hidden--,"b""John Brown's Army""",4,--hidden--,104.03,96.62,97.55,125.84
207,16,b'Wereback',13,--hidden--,b'who invited canada',14,--hidden--,93.21,108.02,91.83,80.24
208,1,b'who invited canada',14,--hidden--,b'Saquon These Nuts',5,--hidden--,111.39,91.02,107.85,73.92
209,2,b'who invited canada',14,--hidden--,"b""Jadeveon's Hit Men""",3,--hidden--,109.15,155.04,103.00,94.90
210,3,b'who invited canada',14,--hidden--,b'SuperCam That Ho',7,--hidden--,106.50,127.70,111.74,159.38
211,4,b'who invited canada',14,--hidden--,b'The Ice Cold Bruschi',1,--hidden--,104.30,95.90,111.66,165.08
212,5,b'who invited canada',14,--hidden--,b'Immortal Duffmen',2,--hidden--,98.00,69.52,96.23,98.86
213,6,b'who invited canada',14,--hidden--,b'The Twelve Apostles',10,--hidden--,112.07,111.00,115.37,136.04


In [15]:
to_date_seasons_df = pd.concat([df, season_matchups_2023_df],ignore_index=True)

In [16]:
to_date_seasons_df.tail()

,week,team_name,season_id,manager_name,opponent_team_name,opponent_season_id,opponent_name,projected_points_for,points_for,projected_points_against,points_against,season
1927,12,b'who invited canada',14,--hidden--,b'Wereback',13,--hidden--,117.32,101.60,94.75,77.66,NaN
1928,13,b'who invited canada',14,--hidden--,b'Starr is Born',6,--hidden--,102.33,69.68,103.41,112.18,NaN
1929,14,b'who invited canada',14,--hidden--,b'Saquon These Nuts',5,--hidden--,103.51,126.10,80.43,79.74,NaN
1930,15,b'who invited canada',14,--hidden--,"b""Jadeveon's Hit Men""",3,--hidden--,109.63,63.28,105.79,113.70,NaN
1931,16,b'who invited canada',14,--hidden--,b'Wereback',13,--hidden--,91.83,80.24,93.21,108.02,NaN


In [100]:
to_date_seasons_df.to_csv('to_date_seasons_raw.csv', index=False)

In [47]:
season_matchups_2022_df = pd.DataFrame(all_season_results)

In [48]:
season_matchups_2022_df

,week,team_name,season_id,nickname,opponent_team_name,opponent_season_id,opponent_nickname,projected_points_for,points_for,projected_points_against,points_against
0,1,b'The Ice Cold Bruschi',1,--hidden--,b'Immortal Duffmen',2,--hidden--,118.13,86.76,113.27,137.28
1,2,b'The Ice Cold Bruschi',1,--hidden--,"b""Jadeveon's Hit Men""",3,--hidden--,114.13,93.84,118.10,130.60
2,3,b'The Ice Cold Bruschi',1,--hidden--,b'Thug Bungalow',11,--hidden--,117.47,129.90,115.91,103.16
3,4,b'The Ice Cold Bruschi',1,--hidden--,b'Thunder Crotches',12,--hidden--,113.29,98.32,109.03,103.20
4,5,b'The Ice Cold Bruschi',1,--hidden--,b'Starr is Born',6,--hidden--,107.05,144.14,105.15,113.68
...,...,...,...,...,...,...,...,...,...,...,...
219,12,b'who invited canada',14,--hidden--,"b""We're Back""",13,--hidden--,120.37,107.50,114.17,92.90
220,13,b'who invited canada',14,--hidden--,b'The Ice Cold Bruschi',1,--hidden--,118.39,126.86,104.28,106.60
221,14,b'who invited canada',14,--hidden--,b'The fishingtons',8,--hidden--,106.21,117.00,123.83,110.56
222,16,b'who invited canada',14,--hidden--,"b""We're Back""",13,--hidden--,114.58,135.56,111.95,76.86
